In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from colorama import Fore
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tld import get_tld, is_tld
from wordcloud import WordCloud

In [3]:
import pandas as pd
df=pd.read_csv('F:\React_projs2\DOM_scraping_major_proj\combined_df.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df


,url,size,serial_no,similarity_score,num_forms,has_login_form,num_scripts,has_external_scripts,num_hyperlinks,num_external_hyperlinks,num_js_events,isPhishing
0,https://beneficioauxbrasil.com,246,1,42.324949,0,0,1,1,0,0,0,1
1,https://secure-ads-manage.miracle-chain.com/bu...,810,2,69.116352,0,0,1,0,0,0,0,1
2,https://pub-3ef4c4f0dccc42ab9f9604f18c4b3622.r...,14108,19,10.240240,0,0,0,0,1,1,0,1
3,https://googleweblight.com/i?u=https://pub-3ef...,14108,20,10.240240,0,0,0,0,1,1,0,1
4,https://00-ird.weebly.com/,34304,22,89.400942,1,0,28,1,5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1664,https://www.auction.co.kr,171679,1029,42.680443,1,0,40,1,224,155,0,0
1665,https://www.bitfinex.com,704630,1030,35.194682,0,0,12,1,92,41,0,0
1666,https://www.viva.co.id,159458,1031,32.729541,0,0,56,1,286,268,0,0
1667,https://www.unity3d.com,293,1032,0.878553,0,0,0,0,0,0,0,0


In [4]:
data=df

In [5]:
def process_tld(url):
    try:
#         Extract the top level domain (TLD) from the URL given
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        pri_domain= res.parsed_url.netloc
    except :
        pri_domain= None
    return pri_domain

feature = ['@','?','-','=','.','#','%','+','$','!','*',',','//']
for a in feature:
    data[a] = data['url'].apply(lambda i: i.count(a))


def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
    
def httpSecure(url):
    htp = urlparse(url).scheme #It supports the following URL schemes: file , ftp , gopher , hdl , 
                               #http , https ... from urllib.parse
    match = str(htp)
    if match=='https':
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
    
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits
data['digits']= data['url'].apply(lambda i: digit_count(i))

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

def Shortining_Service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
    
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4 with port
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '([0-9]+(?:\.[0-9]+){3}:[0-9]+)|'
        '((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)', url)  # Ipv6
    if match:
        return 1
    else:
        return 0
    
data['url_len'] = data['url'].apply(lambda x: len(str(x)))
data['domain'] = data['url'].apply(lambda i: process_tld(i))
data['abnormal_url'] = data['url'].apply(lambda i: abnormal_url(i))
data['https'] = data['url'].apply(lambda i: httpSecure(i))
data['letters']= data['url'].apply(lambda i: letter_count(i))
data['Shortining_Service'] = data['url'].apply(lambda x: Shortining_Service(x))
data['having_ip_address'] = data['url'].apply(lambda i: having_ip_address(i))

In [6]:
data

,url,size,serial_no,similarity_score,num_forms,has_login_form,num_scripts,has_external_scripts,num_hyperlinks,num_external_hyperlinks,...,",",//,digits,url_len,domain,abnormal_url,https,letters,Shortining_Service,having_ip_address
0,https://beneficioauxbrasil.com,246,1,42.324949,0,0,1,1,0,0,...,0,1,0,30,beneficioauxbrasil.com,1,1,26,0,0
1,https://secure-ads-manage.miracle-chain.com/bu...,810,2,69.116352,0,0,1,0,0,0,...,0,1,0,61,secure-ads-manage.miracle-chain.com,1,1,50,0,0
2,https://pub-3ef4c4f0dccc42ab9f9604f18c4b3622.r...,14108,19,10.240240,0,0,0,0,1,1,...,0,1,23,68,pub-3ef4c4f0dccc42ab9f9604f18c4b3622.r2.dev,1,1,37,0,0
3,https://googleweblight.com/i?u=https://pub-3ef...,14108,20,10.240240,0,0,0,0,1,1,...,0,2,23,99,googleweblight.com,1,1,61,1,0
4,https://00-ird.weebly.com/,34304,22,89.400942,1,0,28,1,5,1,...,0,1,2,26,00-ird.weebly.com,1,1,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664,https://www.auction.co.kr,171679,1029,42.680443,1,0,40,1,224,155,...,0,1,0,25,www.auction.co.kr,1,1,19,0,0
1665,https://www.bitfinex.com,704630,1030,35.194682,0,0,12,1,92,41,...,0,1,0,24,www.bitfinex.com,1,1,19,1,0
1666,https://www.viva.co.id,159458,1031,32.729541,0,0,56,1,286,268,...,0,1,0,22,www.viva.co.id,1,1,16,0,0
1667,https://www.unity3d.com,293,1032,0.878553,0,0,0,0,0,0,...,0,1,1,23,www.unity3d.com,1,1,17,0,0


In [13]:
filtered=data[['size','similarity_score','num_forms','num_scripts','num_hyperlinks','num_external_hyperlinks','digits','url_len','has_external_scripts','https','isPhishing']]
filtered

,size,similarity_score,num_forms,num_scripts,num_hyperlinks,num_external_hyperlinks,digits,url_len,has_external_scripts,https,isPhishing
0,246,42.324949,0,1,0,0,0,30,1,1,1
1,810,69.116352,0,1,0,0,0,61,0,1,1
2,14108,10.240240,0,0,1,1,23,68,0,1,1
3,14108,10.240240,0,0,1,1,23,99,0,1,1
4,34304,89.400942,1,28,5,1,2,26,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1664,171679,42.680443,1,40,224,155,0,25,1,1,0
1665,704630,35.194682,0,12,92,41,0,24,1,1,0
1666,159458,32.729541,0,56,286,268,0,22,1,1,0
1667,293,0.878553,0,0,0,0,1,23,0,1,0


In [14]:
X = filtered.drop(['isPhishing'],axis=1)#,'type_code'
y = filtered['isPhishing']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier  # Added XGBoost
from lightgbm import LGBMClassifier  # Added LightGBM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Initialize classifiers
log_reg = LogisticRegression(max_iter=10000)
knn = KNeighborsClassifier()
dtree = DecisionTreeClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()  # Added XGBoost
lgbm = LGBMClassifier()  # Added LightGBM

# Train classifiers
log_reg.fit(X_train, y_train)
knn.fit(X_train, y_train)
dtree.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)  # Added XGBoost
lgbm.fit(X_train, y_train)  # Added LightGBM

# Make predictions for test data
y_pred_log_reg = log_reg.predict(X_test)
y_pred_knn = knn.predict(X_test)
y_pred_dtree = dtree.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_xgb = xgb.predict(X_test)  # Added XGBoost
y_pred_lgbm = lgbm.predict(X_test)  # Added LightGBM

# Make predictions for train data
y_train_pred_log_reg = log_reg.predict(X_train)
y_train_pred_knn = knn.predict(X_train)
y_train_pred_dtree = dtree.predict(X_train)
y_train_pred_rf = rf.predict(X_train)
y_train_pred_xgb = xgb.predict(X_train)  # Added XGBoost
y_train_pred_lgbm = lgbm.predict(X_train)  # Added LightGBM


ModuleNotFoundError: No module named 'lightgbm'

In [25]:
def makeTable(title, y_test, y_pred_log_reg, y_pred_knn, y_pred_dtree, y_pred_rf, y_pred_xgb, y_pred_lgbm):

    metrics = {
        'metrics': ['accuracy', 'precision', 'recall', 'f1_score', 'confusion'],
        'Logistic Regression': [
            accuracy_score(y_test, y_pred_log_reg),
            precision_score(y_test, y_pred_log_reg, average='macro'),
            recall_score(y_test, y_pred_log_reg, average='macro'),
            f1_score(y_test, y_pred_log_reg, average='macro'),
            confusion_matrix(y_test, y_pred_log_reg)
        ],
        'K-Nearest Neighbors': [
            accuracy_score(y_test, y_pred_knn),
            precision_score(y_test, y_pred_knn, average='macro'),
            recall_score(y_test, y_pred_knn, average='macro'),
            f1_score(y_test, y_pred_knn, average='macro'),
            confusion_matrix(y_test, y_pred_knn)
        ],
        'Decision Tree': [
            accuracy_score(y_test, y_pred_dtree),
            precision_score(y_test, y_pred_dtree, average='macro'),
            recall_score(y_test, y_pred_dtree, average='macro'),
            f1_score(y_test, y_pred_dtree, average='macro'),
            confusion_matrix(y_test, y_pred_dtree)
        ],
        'Random Forest': [
            accuracy_score(y_test, y_pred_rf),
            precision_score(y_test, y_pred_rf, average='macro'),
            recall_score(y_test, y_pred_rf, average='macro'),
            f1_score(y_test, y_pred_rf, average='macro'),
            confusion_matrix(y_test, y_pred_rf)
        ],
        'XGBoost': [
            accuracy_score(y_test, y_pred_xgb),
            precision_score(y_test, y_pred_xgb, average='macro'),
            recall_score(y_test, y_pred_xgb, average='macro'),
            f1_score(y_test, y_pred_xgb, average='macro'),
            confusion_matrix(y_test, y_pred_xgb)
        ],
        'LightGBM': [
            accuracy_score(y_test, y_pred_lgbm),
            precision_score(y_test, y_pred_lgbm, average='macro'),
            recall_score(y_test, y_pred_lgbm, average='macro'),
            f1_score(y_test, y_pred_lgbm, average='macro'),
            confusion_matrix(y_test, y_pred_lgbm)
        ]
    }

    # Display results in a tabular format
    results_df = pd.DataFrame(metrics)
    results_df.title = title

    return results_df


In [26]:
table=makeTable("Testing Dataset metrics",y_test, y_pred_log_reg, y_pred_knn, y_pred_dtree, y_pred_rf, y_pred_xgb, y_pred_lgbm)
print(table.title)
table

NameError: name 'y_pred_xgb' is not defined